In [1]:
import numpy as np
from tensorflow import keras
import os
import re
import tensorflow as tf

In [2]:
data = []
for file in os.listdir('./data'):
    with open(f"./data/{file}", encoding="utf-8") as f:
        for line in f:
            data.append(line)

In [3]:
len(data)

479565

In [4]:
data = [i.strip() for i in data]

In [5]:

data = list(filter(None,data))

In [6]:
def clean_data(text):
    newString = text.lower()
    newString = re.sub(r'[^A-Za-z\s]','',newString)
    
    return newString

In [7]:
new = clean_data(data[2])

In [8]:
cleaned_data = [clean_data(text) for text in data]

In [9]:
count = 0
for i in cleaned_data:
    print(i)
    if count > 5:
        break
    count += 1

the project gutenberg ebook three ghost stories by charles dickens
this ebook is for the use of anyone anywhere at no cost and with
almost no restrictions whatsoever  you may copy it give it away or
reuse it under the terms of the project gutenberg license included
with this ebook or online at wwwgutenbergorg
title three ghost stories
author charles dickens


In [10]:
from tensorflow.keras.preprocessing.text import Tokenizer
vocab_size = 5000
tokenizer = Tokenizer(num_words=vocab_size)
tokenizer.fit_on_texts(cleaned_data)
converted = tokenizer.texts_to_sequences(cleaned_data)

In [11]:
import pickle
pickle_open = open("tokenizer.pkl","wb")
pickle.dump(tokenizer,pickle_open)
pickle_open.close()

In [11]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
MAX_LEN = 0
for i in converted:
    if len(i) > MAX_LEN:
        MAX_LEN = len(i)
MAX_LEN

28

In [12]:
converted = pad_sequences(converted, maxlen=MAX_LEN, padding="post")

In [13]:
converted[0]

array([   1,  184,  438, 1183,  205, 1330, 2268,   31,  898, 2500,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0])

In [14]:
count = 0
sequences = []
for i in converted:
    for j in range(1,len(i)-1):
        line = [i[j-1],i[j], i[j+1]]
        sequences.append(line)
        if count > 10000:
            break
        count+=1

In [15]:
sequences = np.array(sequences)

In [17]:
len(sequences)

396991

In [18]:
sequences = sequences[np.all(sequences != 0, axis=1)]

In [19]:
len(sequences)

370215

In [17]:
X, Y = sequences[:,:-1], sequences[:,2]

In [25]:
X

array([[   1,  184],
       [ 184,  438],
       [ 438, 1183],
       ...,
       [1222,   96],
       [ 957,  573],
       [   4,  120]])

In [18]:
from tensorflow.keras.utils import to_categorical
Y = to_categorical(Y, num_classes=vocab_size)

In [22]:
X.shape

(370215, 2)

In [19]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Embedding

In [20]:
dimensions_to_represent_word = 100
 
model = Sequential()
model.add(Embedding(vocab_size, dimensions_to_represent_word, input_length=X.shape[1]))
# We will use a two LSTM hidden layers with 100 memory cells each. 
# More memory cells and a deeper network may achieve better results.
model.add(LSTM(100, return_sequences=True))
model.add(LSTM(100))
model.add(Dense(100, activation='relu'))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 2, 100)            500000    
_________________________________________________________________
lstm (LSTM)                  (None, 2, 100)            80400     
_________________________________________________________________
lstm_1 (LSTM)                (None, 100)               80400     
_________________________________________________________________
dense (Dense)                (None, 100)               10100     
_________________________________________________________________
dense_1 (Dense)              (None, 5000)              505000    
Total params: 1,175,900
Trainable params: 1,175,900
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
model.fit(X, Y, batch_size=128, epochs=100)

Epoch 1/100
3102/3102 [==============================] - 71s 23ms/step - loss: 4.8872 - accuracy: 0.1852
Epoch 2/100
3102/3102 [==============================] - 71s 23ms/step - loss: 4.7384 - accuracy: 0.1962
Epoch 3/100
3102/3102 [==============================] - 70s 23ms/step - loss: 4.6265 - accuracy: 0.2039
Epoch 4/100
3102/3102 [==============================] - 70s 23ms/step - loss: 4.5345 - accuracy: 0.2096
Epoch 5/100
3102/3102 [==============================] - 68s 22ms/step - loss: 4.4548 - accuracy: 0.2136
Epoch 6/100
3102/3102 [==============================] - 68s 22ms/step - loss: 4.3832 - accuracy: 0.2181
Epoch 7/100
3102/3102 [==============================] - 68s 22ms/step - loss: 4.3166 - accuracy: 0.2220
Epoch 8/100
3102/3102 [==============================] - 64s 21ms/step - loss: 4.2550 - accuracy: 0.2250
Epoch 9/100
3102/3102 [==============================] - 52s 17ms/step - loss: 4.1963 - accuracy: 0.2290
Epoch 10/100
3102/3102 [==============================]

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'c:\\users\\naman mistry\\appdata\\local\\programs\\python\\python38\\Lib\\site-packages\\tensorflow\\lite\\experimental\\microfrontend\\python\\ops\\_audio_microfrontend_op.so'
Consider using the `--user` option or check the permissions.



3102/3102 [==============================] - 72s 23ms/step - loss: 3.9482 - accuracy: 0.2449
Epoch 15/100
3102/3102 [==============================] - 59s 19ms/step - loss: 3.9055 - accuracy: 0.2482
Epoch 16/100
3102/3102 [==============================] - 61s 20ms/step - loss: 3.8644 - accuracy: 0.2518
Epoch 17/100
3102/3102 [==============================] - 57s 18ms/step - loss: 3.8254 - accuracy: 0.2550
Epoch 18/100
3102/3102 [==============================] - 54s 17ms/step - loss: 3.7890 - accuracy: 0.2589
Epoch 19/100
3102/3102 [==============================] - 53s 17ms/step - loss: 3.7545 - accuracy: 0.2619
Epoch 20/100
3102/3102 [==============================] - 53s 17ms/step - loss: 3.7201 - accuracy: 0.2657
Epoch 21/100
3102/3102 [==============================] - 53s 17ms/step - loss: 3.6889 - accuracy: 0.2685
Epoch 22/100
3102/3102 [==============================] - 53s 17ms/step - loss: 3.6585 - accuracy: 0.2719
Epoch 23/100
3102/3102 [==============================] - 5

In [32]:
test = ['thank',
'welcome to the',
'when there',
'more than',
'it cannot',
'is that',
'although this',
'do you',
'I was',
'the only',
'a great']

for t in test:
    example = tokenizer.texts_to_sequences([t])
    print(len(example))
    print(example)
    prediction = model.predict(np.array(example))
    predicted_word = np.argmax(prediction)
    reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))  # https://stackoverflow.com/a/43927939/246508
    print ("{0} -> {1}".format(t, reverse_word_map[predicted_word]))

1
[[542]]
thank -> you
1
[[1336, 4, 1]]
welcome to the -> of
1
[[43, 44]]
when there -> was
1
[[64, 79]]
more than -> the
1
[[9, 398]]
it cannot -> be
1
[[24, 8]]
is that -> i
1
[[665, 30]]
although this -> was
1
[[58, 13]]
do you -> know
1
[[7, 12]]
I was -> a
1
[[1, 123]]
the only -> of
1
[[5, 90]]
a great -> and


In [29]:
model.save("model.h5")

In [29]:
!pip install tensorflow

^C


In [30]:
!pip install tensorflow-gpu

In [31]:
import tensorflow
tensorflow.__version__

'2.5.0'